In [13]:
import os
import numpy as np
import pandas as pd
import lightgbm as lgb
import shap
import statsmodels.stats.multitest as multi
from scipy.stats import pearsonr
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kruskal
from matplotlib.ticker import MaxNLocator
import random
from Bio import SeqIO

In [3]:
kmer_info_df = pd.read_csv('/BioII/lulab_b/huangkeyun/zhangys/RNA_locator/python_scripts/SHAP/circRNA_shap_output/kmer_importance_quantification_info.txt',sep = '\t', index_col=0)
kmer_info_df 

,mean_abs_shap,pearsonr_r,pearsonr_p,fdr
AAA,0.038855,0.415940,3.961842e-55,1.850180e-52
AAC,0.059427,-0.670482,3.501988e-169,3.239339e-166
AAG,0.054567,0.773211,4.787443e-257,5.577371e-254
AAT,0.053620,-0.277196,3.461019e-24,1.048689e-21
ACA,0.039691,-0.310678,2.902728e-30,1.004344e-27
...,...,...,...,...
TTTGT,0.047631,0.777141,2.563909e-261,3.004901e-258
TTTTA,0.018986,0.243908,6.312180e-19,1.691664e-16
TTTTC,0.012936,-0.243139,8.183561e-19,2.185011e-16
TTTTG,0.026090,-0.052837,5.779853e-02,9.884977e-01


In [4]:
TOP20_SHAP_kmer = kmer_info_df.sort_values(by = 'mean_abs_shap', ascending=False).head(n = 20)

kmer_info_df['abs_pearsonr'] = kmer_info_df['pearsonr_r'].abs()
TOP20_pearsonr_kmer = kmer_info_df.sort_values(by = 'abs_pearsonr', ascending=False).head(n = 20)
common_index = TOP20_SHAP_kmer.index.intersection(TOP20_pearsonr_kmer.index)
common_index

Index(['CGCA', 'GCACT', 'GAGAC'], dtype='object')

In [11]:
# 获取abs shap kmer最低的10个，作为随机突变目标
LESS10_SHAP_kmer = kmer_info_df.sort_values(by = 'mean_abs_shap', ascending=True).head(n = 10)
LESS20_pearsonr_kmer = kmer_info_df.sort_values(by = 'abs_pearsonr', ascending=True).head(n = 20)
LESS10_SHAP_kmer.index

Index(['CGCAT', 'CGTTA', 'CCGCT', 'TAGGT', 'GCGAT', 'CGCG', 'GCGGT', 'CGTAG',
       'GCGCT', 'CGACG'],
      dtype='object')

In [10]:
TOP20_SHAP_kmer[(TOP20_SHAP_kmer['pearsonr_r'] < 0)].index


Index(['AGT', 'TCTAG', 'TACAG', 'CGCA', 'ATAA', 'GTGA', 'GACC', 'TGGC',
       'AACTT'],
      dtype='object')

In [9]:
TOP20_SHAP_kmer[(TOP20_SHAP_kmer['pearsonr_r'] > 0)].index

Index(['CCAAA', 'GCG', 'AAGAC', 'CTTC', 'AACC', 'ATGGT', 'GCACT', 'GTGC',
       'TGAG', 'GAGAC', 'CGTA'],
      dtype='object')

In [15]:
# 将正类相关kmer突变
input_fasta = "/BioII/lulab_b/huangkeyun/zhangys/RNA_locator/python_scripts/ML_models/circRNA_ML_Model_Output/test_set_sequences1.fasta"
output_fasta = "./test_EV_sequences1_mutated.fasta"

high_shap_kmers = ['CCAAA', 'GCG', 'AAGAC', 'CTTC', 'AACC', 'ATGGT', 'GCACT', 'GTGC','TGAG', 'GAGAC', 'CGTA']
low_shap_kmers = ['CGCAT', 'CGTTA', 'CCGCT', 'TAGGT', 'GCGAT', 'CGCG', 'GCGGT', 'CGTAG', 'GCGCT', 'CGACG'] 

random.seed(198)

# 工具函数：生成非重叠突变坐标
def find_non_overlapping_matches(seq, target_kmers):
    matches = []
    used = set()
    for kmer in target_kmers:
        start = 0
        while True:
            idx = seq.find(kmer, start)
            if idx == -1:
                break
            if all((i not in used) for i in range(idx, idx+len(kmer))):
                matches.append((idx, idx+len(kmer), kmer))
                used.update(range(idx, idx+len(kmer)))
            start = idx + 1
    matches.sort()
    return matches

# 主突变函数
def mutate_sequence(seq_str, high_kmers, low_kmers):
    matches = find_non_overlapping_matches(seq_str, high_kmers)
    seq_list = list(seq_str)
    for start, end, kmer in matches:
        replacement = random.choice(low_kmers)
        # 替换：长度不一致时直接替换start:end为新kmer
        seq_list[start:end] = list(replacement)
    return ''.join(seq_list)

# 读取并突变FASTA
mutated_records = []
for record in SeqIO.parse(input_fasta, "fasta"):
    seq = str(record.seq)
    mutated_seq = mutate_sequence(seq, high_shap_kmers, low_shap_kmers)
    record.seq = mutated_seq
    mutated_records.append(record)

# 保存突变后的FASTA
with open(output_fasta, "w") as f:
    SeqIO.write(mutated_records, f, "fasta")

In [16]:
# 将负类相关kmer突变
input_fasta = "/BioII/lulab_b/huangkeyun/zhangys/RNA_locator/python_scripts/ML_models/circRNA_ML_Model_Output/test_set_sequences1.fasta"
output_fasta = "./test_Cyto_sequences1_mutated.fasta"

high_shap_kmers = ['AGT', 'TCTAG', 'TACAG', 'CGCA', 'ATAA', 'GTGA', 'GACC', 'TGGC','AACTT']
low_shap_kmers = ['CGCAT', 'CGTTA', 'CCGCT', 'TAGGT', 'GCGAT', 'CGCG', 'GCGGT', 'CGTAG', 'GCGCT', 'CGACG'] 

random.seed(198)

# 工具函数：生成非重叠突变坐标
def find_non_overlapping_matches(seq, target_kmers):
    matches = []
    used = set()
    for kmer in target_kmers:
        start = 0
        while True:
            idx = seq.find(kmer, start)
            if idx == -1:
                break
            if all((i not in used) for i in range(idx, idx+len(kmer))):
                matches.append((idx, idx+len(kmer), kmer))
                used.update(range(idx, idx+len(kmer)))
            start = idx + 1
    matches.sort()
    return matches

# 主突变函数
def mutate_sequence(seq_str, high_kmers, low_kmers):
    matches = find_non_overlapping_matches(seq_str, high_kmers)
    seq_list = list(seq_str)
    for start, end, kmer in matches:
        replacement = random.choice(low_kmers)
        # 替换：长度不一致时直接替换start:end为新kmer
        seq_list[start:end] = list(replacement)
    return ''.join(seq_list)

# 读取并突变FASTA
mutated_records = []
for record in SeqIO.parse(input_fasta, "fasta"):
    seq = str(record.seq)
    mutated_seq = mutate_sequence(seq, high_shap_kmers, low_shap_kmers)
    record.seq = mutated_seq
    mutated_records.append(record)

# 保存突变后的FASTA
with open(output_fasta, "w") as f:
    SeqIO.write(mutated_records, f, "fasta")